In [1]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
import numpy as np

2024-03-07 09:41:01.223104: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
# Load dataset
data = pd.read_csv('sample.csv')
data = data[['description','score']]
data['score'] = data['score'].apply(lambda x: x[8:11])

# Clean the 'score' column by removing non-numeric values
data['score'] = data['score'].apply(lambda x: x if x.replace('.', '', 1).isdigit() else None)

# Convert 'score' column to float, handling errors
data['score'] = pd.to_numeric(data['score'], errors='coerce')

# Round 'score' column to integers
data['score'] = data['score'].round().astype('Int64')


data

,description,score
0,Improper Restriction of Excessive Authenticati...,4
1,"Deserialization of Untrusted Data, Improper In...",9
2,An issue was discovered in Samsung Mobile Proc...,8
3,An issue was discovered in Samsung Mobile Proc...,8
4,Cross Site Scripting vulnerability in timetec ...,5
...,...,...
10638,confirmed,9
10639,Chat functionality in Schoolbox application be...,9
10640,News functionality in Schoolbox application be...,7
10641,Class functionality in Schoolbox application \...,7


In [16]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Initialize an empty list to store preprocessed tokens
preprocessed_tokens = []

# Iterate through each description in the DataFrame
for i in range(len(data)):
    text = data['description'][i]
    # Input text with extra whitespace

    # Step 1: Remove leading and trailing white space and replace multiple consecutive white space characters with a single space
    text = " ".join(text.split())

    # Step 2: Tokenize the text
    tokens = word_tokenize(text)

    # Step 3: Lowercase the tokens
    lowercased_tokens = [token.lower() for token in tokens]

    # Step 4: Remove punctuation
    tokens_without_punctuation = [token for token in lowercased_tokens if token not in string.punctuation]

    # Step 5: Remove stopwords
    stop_words = set(stopwords.words("english"))
    filtered_tokens = [token for token in tokens_without_punctuation if token not in stop_words]

    # Append the list of preprocessed tokens to the initialized empty list
    preprocessed_tokens.append(filtered_tokens)

# Assign the list of preprocessed tokens to the new column 'descriptiontoken' in the DataFrame
data['descriptiontoken'] = preprocessed_tokens


In [17]:
data

,description,score,descriptiontoken
0,Improper Restriction of Excessive Authenticati...,4,"[improper, restriction, excessive, authenticat..."
1,"Deserialization of Untrusted Data, Improper In...",9,"[deserialization, untrusted, data, improper, i..."
2,An issue was discovered in Samsung Mobile Proc...,8,"[issue, discovered, samsung, mobile, processor..."
3,An issue was discovered in Samsung Mobile Proc...,8,"[issue, discovered, samsung, mobile, processor..."
4,Cross Site Scripting vulnerability in timetec ...,5,"[cross, site, scripting, vulnerability, timete..."
...,...,...,...
10638,confirmed,9,[confirmed]
10639,Chat functionality in Schoolbox application be...,9,"[chat, functionality, schoolbox, application, ..."
10640,News functionality in Schoolbox application be...,7,"[news, functionality, schoolbox, application, ..."
10641,Class functionality in Schoolbox application \...,7,"[class, functionality, schoolbox, application,..."


In [28]:
# Feature Extraction
data.dropna(subset=['score'], inplace=True)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['description'])
y = data['score']

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert sparse matrix to NumPy array
X_train = X_train.toarray()

# Convert labels to numerical format
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [29]:
X

<9287x18260 sparse matrix of type '<class 'numpy.float64'>'
	with 325578 stored elements in Compressed Sparse Row format>

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Instantiate the logistic regression model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5613562970936491


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [35]:
import joblib

# Load the saved model from the file
joblib.dump(model, 'modelo.pkl')

['modelo.pkl']